# Introducción

Este cuaderno es para desarrollar el Challenge Telecom X de Alura Latam. El desafío es recopilar, procesar y analizar los datos, utilizando Python y sus principales bibliotecas para extraer información que facilite al equipo de Data Science a avanzar en modelos predictivos y desarrollar estrategias para reducir la evasión de clientes.

#📌 Extracción

# 1.Importación de bibliotecas

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import chi2_contingency
import requests
import json
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# 2.Carga del dataset

In [4]:
url = "https://raw.githubusercontent.com/maximo-he/Challenge-Telecom-X/refs/heads/main/TelecomX_Data.json"

try:
    response = requests.get(url)
    response.raise_for_status() # Raise an exception for bad status codes

    # Parse the JSON content
    data = json.loads(response.text)

    print("Data extracted successfully!")
    # You can now work with the 'data' variable which contains the parsed JSON
    # print(data) # Uncomment to see the extracted data

except requests.exceptions.RequestException as e:
    print(f"Error fetching data: {e}")
except json.JSONDecodeError as e:
    print(f"Error decoding JSON: {e}")

Data extracted successfully!


In [5]:
try:
    df = pd.DataFrame(data)
    print("DataFrame created successfully!")

except Exception as e:
    print(f"Error creating DataFrame: {e}")

DataFrame created successfully!


In [6]:
df.head()

,customerID,Churn,customer,phone,internet,account
0,0002-ORFBO,No,"{'gender': 'Female', 'SeniorCitizen': 0, 'Part...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'One year', 'PaperlessBilling': '..."
1,0003-MKNFE,No,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'Yes'}","{'InternetService': 'DSL', 'OnlineSecurity': '...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
2,0004-TLHLJ,Yes,"{'gender': 'Male', 'SeniorCitizen': 0, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
3,0011-IGKFF,Yes,"{'gender': 'Male', 'SeniorCitizen': 1, 'Partne...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."
4,0013-EXCHZ,Yes,"{'gender': 'Female', 'SeniorCitizen': 1, 'Part...","{'PhoneService': 'Yes', 'MultipleLines': 'No'}","{'InternetService': 'Fiber optic', 'OnlineSecu...","{'Contract': 'Month-to-month', 'PaperlessBilli..."


In [7]:
# normalizar
customer_df = pd.json_normalize(df['customer'])
phone_df = pd.json_normalize(df['phone'])
internet_df = pd.json_normalize(df['internet'])
account_df = pd.json_normalize(df['account'])

# concatenar
df = pd.concat([df.drop(['customer', 'phone', 'internet', 'account'], axis=1), customer_df, phone_df, internet_df,account_df], axis=1)

df.head()

,customerID,Churn,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,...,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,Charges.Monthly,Charges.Total
0,0002-ORFBO,No,Female,0,Yes,Yes,9,Yes,No,DSL,...,Yes,No,Yes,Yes,No,One year,Yes,Mailed check,65.6,593.3
1,0003-MKNFE,No,Male,0,No,No,9,Yes,Yes,DSL,...,No,No,No,No,Yes,Month-to-month,No,Mailed check,59.9,542.4
2,0004-TLHLJ,Yes,Male,0,No,No,4,Yes,No,Fiber optic,...,No,Yes,No,No,No,Month-to-month,Yes,Electronic check,73.9,280.85
3,0011-IGKFF,Yes,Male,1,Yes,No,13,Yes,No,Fiber optic,...,Yes,Yes,No,Yes,Yes,Month-to-month,Yes,Electronic check,98.0,1237.85
4,0013-EXCHZ,Yes,Female,1,Yes,No,3,Yes,No,Fiber optic,...,No,No,Yes,Yes,No,Month-to-month,Yes,Mailed check,83.9,267.4


#🔧 Transformación

# 3. Limpieza y preprocesamiento de datos

In [1]:
## 1. Carga y Limpieza de Datos
def load_and_clean_data(filepath):
    """Carga y limpia el dataset"""
    df = pd.read_csv(filepath)

    # Limpieza básica
    df_clean = df.copy()
    df_clean['Churn'] = df_clean['Churn'].map({'Yes': 1, 'No': 0})
    df_clean['SeniorCitizen'] = df_clean['SeniorCitizen'].map({0: 'No', 1: 'Yes'})

    # Eliminar columnas no necesarias
    cols_to_drop = ['customerID']
    if 'Unnamed: 0' in df_clean.columns:
        cols_to_drop.append('Unnamed: 0')
    df_clean = df_clean.drop(columns=cols_to_drop, errors='ignore')

    return df_clean

#📊 Carga y análisis

# 4.Análisis de datos

In [2]:
#visualiza distribuciones
def cramers_v(x, y):
    """Calcula el coeficiente V de Cramer para variables categóricas"""
    confusion_matrix = pd.crosstab(x, y)
    chi2 = chi2_contingency(confusion_matrix)[0]
    n = confusion_matrix.sum().sum()
    phi2 = chi2/n
    r, k = confusion_matrix.shape
    phi2corr = max(0, phi2 - ((k-1)*(r-1))/(n-1))
    rcorr = r - ((r-1)**2)/(n-1)
    kcorr = k - ((k-1)**2)/(n-1)
    return np.sqrt(phi2corr / min((kcorr-1), (rcorr-1)))

In [3]:
#correlación variables numércias y categóricas
def plot_distributions(df, columns, hue='Churn', max_cols=3):
    """Visualiza distribuciones de variables categóricas"""
    n_plots = len(columns)
    n_rows = (n_plots + max_cols - 1) // max_cols

    plt.figure(figsize=(18, 5*n_rows))
    for i, col in enumerate(columns, 1):
        plt.subplot(n_rows, max_cols, i)
        sns.countplot(data=df_clean, x=col, hue=hue)
        plt.title(f'{col} vs Churn')
        plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

In [4]:
#visualiza relación entre variables y variable churn
def plot_correlations(df, num_cols, cat_cols):
    """Visualiza correlaciones numéricas y categóricas"""
    # Correlación numérica
    plt.figure(figsize=(10, 8))
    sns.heatmap(df_clean[num_cols].corr(), annot=True, cmap='coolwarm', fmt='.2f', center=0)
    plt.title('Correlación entre Variables Numéricas')
    plt.show()

    # Correlación categórica
    cramer_matrix = pd.DataFrame(index=cat_cols, columns=cat_cols, dtype=float)
    for col1 in cat_cols:
        for col2 in cat_cols:
            cramer_matrix.loc[col1, col2] = cramers_v(df[col1], df[col2])

    plt.figure(figsize=(14, 12))
    sns.heatmap(cramer_matrix, annot=True, cmap='coolwarm', fmt='.2f', vmin=0, vmax=1)
    plt.title('Correlación Categórica (V de Cramer)')
    plt.show()

In [ ]:
#análisis exploratorio de datos
def plot_numerical_vs_churn(df, num_cols):
    """Visualiza la relación entre variables numéricas y churn"""
    plt.figure(figsize=(16, 4))
    for i, col in enumerate(num_cols, 1):
        plt.subplot(1, len(num_cols), i)
        sns.boxplot(data=df_clean, x='Churn', y=col)
        plt.title(f'{col} vs Churn')
    plt.tight_layout()
    plt.show()

In [6]:
# Variables clave
cat_cols = ['gender', 'SeniorCitizen', 'Partner', 'Dependents',
           'OnlineSecurity', 'TechSupport',
           'PaperlessBilling', 'PaymentMethod']

df_clean = df.copy()
# Verifica si 'Churn' es categórica 'Yes'/'No' o numérica 0/1 antes de mapear
if df_clean['Churn'].dtype == 'object':
     df_clean['Churn'] = df_clean['Churn'].map({'Yes': 1, 'No': 0})
# Verifica si 'SeniorCitizen' es numérica 0/1 o categórica 'Yes'/'No'
if df_clean['SeniorCitizen'].dtype != 'object':
    df_clean['SeniorCitizen'] = df_clean['SeniorCitizen'].map({0: 'No', 1: 'Yes'})

# Eliminar columnas no necesarias - asegurate de que customerID exista si lo intentas eliminar
cols_to_drop = []
if 'customerID' in df_clean.columns:
    cols_to_drop.append('customerID')
if 'Unnamed: 0' in df_clean.columns: # Check if this column exists after JSON normalization
    cols_to_drop.append('Unnamed: 0')
if cols_to_drop:
    df_clean = df_clean.drop(columns=cols_to_drop, errors='ignore')


# Visualizaciones
plot_distributions(df_clean, cat_cols)
plot_numerical_vs_churn(df_clean, num_cols)
plot_correlations(df_clean, num_cols, cat_cols[:8])

NameError: name 'df' is not defined

In [7]:
def analyze_segments(df_clean, segment_cols, target='Churn', palette_name='husl'):
    """Analiza tasas de churn por segmentos con estilo minimalista

    Args:
        df (pd.DataFrame): DataFrame con los datos
        segment_cols (list): Lista de columnas para segmentar
        target (str): Columna objetivo (default 'Churn')
        palette_name (str): Nombre de la paleta de colores (default 'husl')
    """
    # Configuración de estilo minimalista
    sns.set(style="white")  # Elimina fondos y líneas innecesarias

    for col in segment_cols:
        # Crear figura sin márgenes superiores y derechos
        fig, ax = plt.subplots(figsize=(10, 5))

        # Calcular datos para el gráfico
        segment_data = df_clean.groupby(col)[target].mean().sort_values(ascending=False)

        # Crear paleta de colores
        custom_palette = sns.color_palette(palette_name, len(segment_data.index))

        # Crear gráfico de barras
        sns.barplot(x=segment_data.index,
                   y=segment_data.values,
                   palette=custom_palette,
                   ax=ax)

        # Personalización minimalista
        ax.set_title(f'Tasa de Churn por {col}', fontsize=14, pad=15)
        ax.set_ylabel('Tasa de Churn', fontsize=12)
        ax.set_xlabel('')

        # Añadir valores en las barras
        for p in ax.patches:
            ax.annotate(f'{p.get_height():.1%}',
                       (p.get_x() + p.get_width() / 2., p.get_height()),
                       ha='center', va='center',
                       xytext=(0, 5),
                       textcoords='offset points',
                       fontsize=10)

        # Eliminar bordes/márgenes superiores y derechos
        ax.spines['right'].set_visible(False)
        ax.spines['top'].set_visible(False)

        # Ajustar layout para minimizar espacios en blanco
        plt.tight_layout()
        plt.show()

        # Mostrar tabla con conteos (formato mejorado)
        print(f"\nDistribución y churn por {col}:")
        display(pd.concat([
            df_clean[col].value_counts(normalize=True).rename('Distribución').mul(100).round(1).astype(str) + '%',
            df_clean.groupby(col)[target].mean().rename('Tasa Churn').mul(100).round(1).astype(str) + '%'
        ], axis=1).sort_values('Tasa Churn', ascending=False))
plt.show()

NameError: name 'plt' is not defined

#📄Informe final

El análisis del comportamiento de los clientes revela patrones claros que explican las tasas de abandono (churn) en el servicio. Los hallazgos principales muestran que el churn no ocurre aleatoriamente, sino que está fuertemente influenciado por factores específicos que interactúan entre sí de maneras predecibles. La combinación de contratos a corto plazo, falta de servicios digitales esenciales y métodos de pago inconvenientes crea una "tormenta perfecta" para la pérdida de clientes.

Un hallazgo clave es el poder estabilizador de los contratos a largo plazo. Los clientes con contratos anuales o bianuales muestran tasas de churn significativamente menores (menos del 10%), independientemente de otros factores. Este efecto protector se potencia cuando se combina con la contratación de servicios de soporte técnico y seguridad en línea. Por otro lado, los clientes con contratos mensuales que carecen de estos servicios digitales presentan tasas de churn alarmantemente altas, superando el 40% en algunos segmentos.

El análisis también descubrió diferencias importantes en el comportamiento entre segmentos demográficos. Los clientes senior, aunque representan un porcentaje menor de la base total, muestran mayor propensión al abandono durante su primer año, pero se vuelven notablemente más leales después de superar este período inicial. Esto sugiere que los esfuerzos de retención deben adaptarse específicamente a las necesidades y preocupaciones de este grupo demográfico durante sus primeros meses de servicio.